In [1]:
import tensorflow as tf

IMG_SIZE = 224
BATCH_SIZE = 32

train_path = "/content/drive/MyDrive/dog emotion.v2i.multiclass/train"
val_path   = "/content/drive/MyDrive/dog emotion.v2i.multiclass/val"
test_path  = "/content/drive/MyDrive/dog emotion.v2i.multiclass/test"

# ------------------------
# Load Train Dataset
# ------------------------
train_ds= tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

# ------------------------
# Load Validation Dataset
# ------------------------
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_path,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

# ------------------------
# Load Test Dataset
# ------------------------
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Get class names before prefetching
class_names = train_ds.class_names
print("Classes:", class_names)

# Auto prefetching for speed
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)


Found 12510 files belonging to 4 classes.
Found 3574 files belonging to 4 classes.
Found 1787 files belonging to 4 classes.
Classes: ['angry', 'happy', 'relaxed', 'sad']


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

# Load base model
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False   # First freeze

# Build model
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = tf.keras.applications.efficientnet.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(4, activation='softmax')(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         5,124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 5,124 (20.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [3]:
history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)


Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 2435s 6s/step - accuracy: 0.4063 - loss: 1.2840 - val_accuracy: 0.5823 - val_loss: 1.0064
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 170ms/step - accuracy: 0.5708 - loss: 1.0174 - val_accuracy: 0.6198 - val_loss: 0.9301
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 72s 185ms/step - accuracy: 0.6154 - loss: 0.9375 - val_accuracy: 0.6363 - val_loss: 0.8953
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 184ms/step - accuracy: 0.6192 - loss: 0.9146 - val_accuracy: 0.6480 - val_loss: 0.8754
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 168ms/step - accuracy: 0.6298 - loss: 0.8972 - val_accuracy: 0.6511 - val_loss: 0.8600
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 168ms/step - accuracy: 0.6383 - loss: 0.8820 - val_accuracy: 0.6570 - val_loss: 0.8486
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 167ms/step - accuracy: 0.6416 - loss: 0.8679 - val_accuracy: 0.6648 - val_loss: 0.8396
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 168ms/step - accuracy: 0.6534 - loss: 0.

In [4]:
base_model.trainable = True

for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25
)


Epoch 1/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 125s 237ms/step - accuracy: 0.5868 - loss: 0.9907 - val_accuracy: 0.6561 - val_loss: 0.8512
Epoch 2/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 169ms/step - accuracy: 0.6327 - loss: 0.8833 - val_accuracy: 0.6726 - val_loss: 0.8101
Epoch 3/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 170ms/step - accuracy: 0.6545 - loss: 0.8222 - val_accuracy: 0.6844 - val_loss: 0.7796
Epoch 4/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 67s 170ms/step - accuracy: 0.6681 - loss: 0.7898 - val_accuracy: 0.6911 - val_loss: 0.7558
Epoch 5/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 67s 171ms/step - accuracy: 0.6891 - loss: 0.7558 - val_accuracy: 0.7043 - val_loss: 0.7341
Epoch 6/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 170ms/step - accuracy: 0.7048 - loss: 0.7191 - val_accuracy: 0.7090 - val_loss: 0.7169
Epoch 7/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 67s 172ms/step - accuracy: 0.7130 - loss: 0.7034 - val_accuracy: 0.7154 - val_loss: 0.7011
Epoch 8/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 67s 172ms/step - accuracy: 0.7278 - loss: 

In [5]:
base_model.trainable = True

# Unfreeze last 30 layers of EfficientNet
for layer in base_model.layers[:-60]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)


Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 139s 279ms/step - accuracy: 0.8549 - loss: 0.3868 - val_accuracy: 0.7848 - val_loss: 0.5495
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 170ms/step - accuracy: 0.8587 - loss: 0.3784 - val_accuracy: 0.7882 - val_loss: 0.5473
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 81s 168ms/step - accuracy: 0.8606 - loss: 0.3700 - val_accuracy: 0.7893 - val_loss: 0.5417
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 168ms/step - accuracy: 0.8685 - loss: 0.3555 - val_accuracy: 0.7876 - val_loss: 0.5392
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 168ms/step - accuracy: 0.8738 - loss: 0.3403 - val_accuracy: 0.7907 - val_loss: 0.5351
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 72s 185ms/step - accuracy: 0.8761 - loss: 0.3318 - val_accuracy: 0.7941 - val_loss: 0.5324
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 75s 168ms/step - accuracy: 0.8823 - loss: 0.3216 - val_accuracy: 0.7935 - val_loss: 0.5283
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 169ms/step - accuracy: 0.8827 - loss: 

In [ ]:
base_model.trainable = True

# Unfreeze last 30 layers of EfficientNet
for layer in base_model.layers[:-70]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)


Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 119s 230ms/step - accuracy: 0.9137 - loss: 0.2486 - val_accuracy: 0.7988 - val_loss: 0.5176
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 168ms/step - accuracy: 0.9140 - loss: 0.2417 - val_accuracy: 0.8025 - val_loss: 0.5178
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 65s 167ms/step - accuracy: 0.9151 - loss: 0.2401 - val_accuracy: 0.8044 - val_loss: 0.5186
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 65s 167ms/step - accuracy: 0.9186 - loss: 0.2347 - val_accuracy: 0.8050 - val_loss: 0.5192
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 65s 166ms/step - accuracy: 0.9197 - loss: 0.2278 - val_accuracy: 0.8041 - val_loss: 0.5201
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 83s 168ms/step - accuracy: 0.9242 - loss: 0.2180 - val_accuracy: 0.8053 - val_loss: 0.5157
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 169ms/step - accuracy: 0.9195 - loss: 0.2237 - val_accuracy: 0.8067 - val_loss: 0.5167
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 66s 168ms/step - accuracy: 0.9246 - loss: 

In [1]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")

NameError: name 'model' is not defined

In [7]:
model.save("/content/drive/MyDrive/dog emotion.v2i.multiclass/final_model.h5")
print("Model saved successfully!")


Model saved successfully!


In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# 1️⃣ Load your saved model
model = tf.keras.models.load_model("/content/drive/MyDrive/dog emotion.v2i.multiclass/final_model.h5")

# 2️⃣ Set image size (same as training)
IMG_SIZE = 224

# 3️⃣ Class names (replace with your 4 classes exactly)
class_names = ["angry", "happy", "relaxed", "sad"]

# 4️⃣ Prediction function
def predict_emotion(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

    # Preprocess for EfficientNet
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)

    # Predict
    predictions = model.predict(img_array)
    class_id = np.argmax(predictions)
    confidence = np.max(predictions)

    return class_names[class_id], float(confidence)

# 5️⃣ Test with your image
img_path = "/content/drive/MyDrive/dog emotion.v2i.multiclass/test/happy/156079418_07d237487e_b_jpg.rf.2831493a4e4c4d0fd883f970ded062e1.jpg"
emotion, confidence = predict_emotion(img_path)

print(f"Predicted Emotion: {emotion}")
print(f"Confidence: {confidence:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Emotion: happy
Confidence: 0.77
